In [1]:
import rlcard
import torch
import pathlib
import numpy
import ray
import models
import ddz

import self_play_test
import time
from rlcard.agents import RandomAgent
from rlcard.utils import set_seed, tournament

In [2]:
config = ddz.MuZeroConfig()
observation_shapes = config.observation_shapes
configs = []
for i in range(config.num_players):
    config = ddz.MuZeroConfig()
    config.observation_shape = observation_shapes[i]
    configs.append(config)

In [3]:
agents = []
# device = torch.device('cpu')
for i in range(config.num_players):
    model = models.MuZeroNetwork(configs[i])
#     model = model.to(device)
    model = model.cuda()
    model.eval()
#     print(next(model.parameters()).device)
    agents.append(model)

ray.init(num_gpus=1.0, ignore_reinit_error=True)

2022-10-17 19:15:44,520	INFO worker.py:1515 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.7.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


In [4]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     numpy.random.seed(seed)
     torch.backends.cudnn.deterministic = True

landlord_paths = '/home/yg/code/Muzero//muzero/results/test_ddz/new/model0.checkpoint'
farmer0_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model1.checkpoint'
farmer1_paths = '/home/yg/code/Muzero/muzero/results/test_ddz/new/model2.checkpoint'
new_model = [landlord_paths, farmer0_paths, farmer1_paths]

landlord_paths_o = '/home/yg/code/Muzero//muzero/results/test_ddz/old/model0.checkpoint'
farmer0_paths_o = '/home/yg/code/Muzero/muzero/results/test_ddz/old/model1.checkpoint'
farmer1_paths_o = '/home/yg/code/Muzero/muzero/results/test_ddz/old/model2.checkpoint'
old_model = [landlord_paths_o, farmer0_paths_o, farmer1_paths_o]
order = [[new_model[0],new_model[1],new_model[2]],[new_model[0],old_model[1],old_model[2]],
         [old_model[0],new_model[1],new_model[2]],[old_model[0],old_model[1],old_model[2]]]

four_res = []
for i in range(1, len(order)):
    models_paths = order[i]

# models_paths = [landlord_paths,farmer0_paths,farmer1_paths]
# map_location = torch.device('cpu')
    for i in range(3):
        model_path = pathlib.Path(models_paths[i])
        model_checkpoint = torch.load(model_path)
        agents[i].set_weights(model_checkpoint['weights'])
        
    seed = 42
    setup_seed(seed)
    num_test = 100
    env = ddz.Game(seed)
    env.set_agents(agents)
    
    self_play_worker = self_play_test.SelfPlayTest.options(
                            num_cpus=0,
                            num_gpus=1.0,
                        ).remote(env, config, configs, seed, agents)

    results = []
    for i in range(num_test):
        start_time = time.time()
        results.append(
            ray.get(
                self_play_worker.play_game.remote(
                    0,
                    0,
                    False,
                    "self",
                    0,
                )
            )
        )
        end_time = time.time()
        print('time',end_time-start_time, i)
    
    res_last = []
    result = [[],[],[]]
    for game_historys in results:
        for i in range(3):
            result[i].append(
                sum(
                    reward_history
                    for k, reward in enumerate(game_historys[i].reward_history)
                )
            )
    for i in range(3):
        res_last.append(numpy.mean(result[i]))
    print(res_last)
    four_res.append(res_last)

set_seed------------
time 31.112853288650513 0
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['S3 D4 DQ H2', '', 'C3 D3 C4 D5 S6 C7 H7 C9 S9 HT DJ CK HK DA C2 D2'], 'hand_cards': ['34Q2', '', '334567799TJKKA22'], 'trace': [(0, 'AAA2'), (1, 'pass'), (2, 'pass'), (0, '5888'), (1, 'pass'), (2, 'pass'), (0, '55'), (1, '66'), (2, 'pass'), (0, 'pass'), (1, 'TTT'), (2, 'pass'), (0, 'pass'), (1, 'QQ'), (2, 'pass'), (0, 'pass'), (1, '3'), (2, '8'), (0, 'K'), (1, 'B'), (2, 'pass'), (0, 'R'), (1, 'pass'), (2, 'pass'), (0, '7'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, '7'), (2, 'pass'), (0, 'J'), (1, 'pass'), (2, 'pass'), (0, '99'), (1, 'JJ'), (2, 'pass'), (0, 'pass'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, '44'), (2, 'pass'), (0, 'pass'), (1, '6')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['ST', '', 'C3 S3 H4 S4 D5 S6 H7 D8 DT SJ CQ HQ CK DK CA HA'], 'hand_cards': ['T', '', '33445678TJQQKKAA'], 'trace': [(0, '9

time(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['H3 S6 HJ', '', 'S3 C3 H5 D7 C7 H7 C9 HT DT CT DJ CJ HQ HK DK C2 H2'], 'hand_cards': ['36J', '', '3357779TTTJJQKK22'], 'trace': [(0, '3AAA'), (1, 'BR'), (2, 'pass'), (0, 'pass'), (1, '66'), (2, 'pass'), (0, 'pass'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, '9'), (2, 'pass'), (0, 'Q'), (1, '2'), (2, 'pass'), (0, 'pass'), (1, '888A'), (2, 'pass'), (0, 'pass'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, 'Q'), (2, 'pass'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '6789T'), (1, 'pass'), (2, 'pass'), (0, 'J'), (1, 'pass'), (2, 'pass'), (0, '55'), (1, 'pass'), (2, 'pass'), (0, '4'), (1, 'pass'), (2, 'pass'), (0, '4'), (1, 'pass'), (2, 'pass'), (0, '9'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, '5'), (2, 'pass'), (0, 'pass'), (1, '44')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
 22.784318447113037 9
time 23.979053497314453 10
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['D3 DT', '', 'C3 D5 H6 D8 C8 S8 S9 H9 S

time 12.872992992401123 19
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['H4 D5 H6 D8 D9 S9 CT CJ C2', '', 'C5 H5 S5 D7 S7 H8 C9 H9 DT ST SJ DK HK S2 D2 H2 BJ'], 'hand_cards': ['456899TJ2', '', '55577899TTJKK222B'], 'trace': [(0, 'JKAAAA'), (1, 'pass'), (2, 'pass'), (0, '8'), (1, 'R'), (2, 'pass'), (0, 'pass'), (1, '3334448J'), (2, 'pass'), (0, 'pass'), (1, '7'), (2, 'pass'), (0, 'pass'), (1, '7'), (2, 'pass'), (0, 'Q'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, '66'), (2, 'pass'), (0, 'QQ'), (1, 'pass'), (2, 'pass'), (0, '3'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, 'T'), (2, 'pass'), (0, 'pass'), (1, '6')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
time 24.39779257774353 20
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['', 'S4 C5 C6 H8 D8', 'S3 S5 H5 H6 D6 S6 H7 C7 S9 D9 DJ CJ HJ HK HA RJ'], 'hand_cards': ['', '45688', '3556667799JJJKAR'], 'trace': [(0, '3TTTTQ'), (1, 'pass'), (2, 'pass'), (0, 'K'), (1, 'A'), (2, 'pass'), (0, 'pass'), (1, '7'), (

time 20.63783621788025 29
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['C3 DT CT DQ', '', 'D3 C4 H5 S6 S7 C8 S8 H8 DJ HJ DK CA DA D2 C2 BJ RJ'], 'hand_cards': ['3TTQ', '', '34567888JJKAA22BR'], 'trace': [(0, '9'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, '66'), (2, 'pass'), (0, 'KK'), (1, 'pass'), (2, 'pass'), (0, '4777'), (1, 'pass'), (2, 'pass'), (0, '6'), (1, '8'), (2, 'pass'), (0, 'Q'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, '33'), (2, 'pass'), (0, '44'), (1, 'TT'), (2, 'pass'), (0, 'pass'), (1, '5'), (2, 'pass'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '5'), (1, '9'), (2, 'pass'), (0, 'J'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, 'J'), (2, 'pass'), (0, 'pass'), (1, '9'), (2, 'pass'), (0, 'A'), (1, '2'), (2, 'pass'), (0, 'pass'), (1, '5'), (2, 'pass'), (0, '9'), (1, 'A')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['S5', '', 'H3 H4 C4 H6 D6 C7 S7 C8 CT HJ CJ DJ HQ SQ CA S2 H2'], 'hand_cards': ['5'

time 22.608851194381714 39
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['D6 C8 CA', '', 'S3 H4 H5 C5 D5 C6 S6 H7 D8 DT HQ DQ DK HK SA D2'], 'hand_cards': ['68A', '', '345556678TQQKKA2'], 'trace': [(0, '4999TTTK'), (1, 'pass'), (2, 'pass'), (0, '33'), (1, '22'), (2, 'pass'), (0, 'pass'), (1, '3'), (2, 'J'), (0, 'pass'), (1, 'R'), (2, 'pass'), (0, 'pass'), (1, '7'), (2, 'pass'), (0, 'pass'), (1, '5'), (2, 'pass'), (0, '7'), (1, '8'), (2, 'pass'), (0, 'B'), (1, 'pass'), (2, 'pass'), (0, 'J'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, '9'), (2, 'pass'), (0, 'K'), (1, '2'), (2, 'pass'), (0, 'pass'), (1, 'Q'), (2, 'pass'), (0, 'A'), (1, 'pass'), (2, 'pass'), (0, '7'), (1, 'J'), (2, 'pass'), (0, 'pass'), (1, 'J'), (2, 'pass'), (0, 'pass'), (1, '4'), (2, 'pass'), (0, 'pass'), (1, '4'), (2, 'pass'), (0, '8'), (1, 'A'), (2, 'pass'), (0, 'pass'), (1, '6')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
time 21.26720142364502 40
(SelfPlayTest pid=3622) {'hand_cards_w

...

time 18.125327587127686 69
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['D4', '', 'C3 S4 C7 C8 H8 S9 CT HT CJ SJ CQ SQ HQ HK CK RJ'], 'hand_cards': ['4', '', '347889TTJJQQQKKR'], 'trace': [(0, '55666'), (1, '77AAA'), (2, 'pass'), (0, '99222'), (1, 'pass'), (2, 'pass'), (0, '789TJ'), (1, 'pass'), (2, 'pass'), (0, 'A'), (1, 'pass'), (2, 'pass'), (0, '33'), (1, '44'), (2, 'pass'), (0, 'pass'), (1, '3'), (2, '8'), (0, 'pass'), (1, '2'), (2, 'pass'), (0, 'pass'), (1, '5'), (2, 'pass'), (0, 'Q'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, 'K'), (2, 'pass'), (0, 'pass'), (1, '5'), (2, 'pass'), (0, 'pass'), (1, '6'), (2, 'pass'), (0, 'pass'), (1, 'B'), (2, 'pass'), (0, 'pass'), (1, 'T'), (2, 'pass'), (0, 'pass'), (1, 'J')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
time(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['S5 D6', '', 'H3 H4 S7 D7 D8 C9 H9 D9 DT CJ HJ DQ CK SK HK S2 RJ'], 'hand_cards': ['56', '', '34778999TJJQKKK2R'], 'trace': [(0, '6QQQ'), (1, 'p

time 25.398409128189087 79
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['H3 C4 C5', '', 'D3 D4 D5 H6 H7 H8 C9 DT DJ HJ CK DA CA HA H2 RJ'], 'hand_cards': ['345', '', '3456789TJJKAAA2R'], 'trace': [(0, 'Q'), (1, 'K'), (2, '2'), (0, 'pass'), (1, 'B'), (2, 'pass'), (0, 'pass'), (1, '7'), (2, 'pass'), (0, '8'), (1, 'pass'), (2, 'pass'), (0, '789TJQK'), (1, 'pass'), (2, 'pass'), (0, '8'), (1, '9'), (2, 'pass'), (0, 'K'), (1, 'pass'), (2, 'pass'), (0, '3'), (1, '9'), (2, 'pass'), (0, 'pass'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, 'T'), (2, 'pass'), (0, 'pass'), (1, 'Q'), (2, 'pass'), (0, 'pass'), (1, 'J'), (2, 'pass'), (0, 'pass'), (1, '3'), (2, 'pass'), (0, 'pass'), (1, 'T'), (2, 'pass'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '7'), (1, '2'), (2, 'pass'), (0, 'pass'), (1, '55'), (2, 'pass'), (0, '66'), (1, 'pass'), (2, 'pass'), (0, '4'), (1, '6'), (2, 'pass'), (0, 'pass'), (1, 'A'), (2, 'pass'), (0, 'pass'), (1, '4')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622

time 20.758042812347412 88
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['S3 H8 HT SK', '', 'H3 D3 C4 D4 H5 D8 S8 D9 C9 H9 CJ HJ HK DA CA D2 C2'], 'hand_cards': ['38TK', '', '3344588999JJKAA22'], 'trace': [(0, 'TTQQQQ'), (1, 'pass'), (2, 'pass'), (0, '66'), (1, '77'), (2, 'pass'), (0, 'BR'), (1, 'pass'), (2, 'pass'), (0, '5'), (1, '7'), (2, 'pass'), (0, 'A'), (1, '2'), (2, 'pass'), (0, 'pass'), (1, '66'), (2, 'pass'), (0, 'pass'), (1, 'K'), (2, 'pass'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '5'), (1, '8'), (2, 'pass'), (0, 'pass'), (1, '5'), (2, 'pass'), (0, 'pass'), (1, '3'), (2, 'pass'), (0, 'pass'), (1, 'JJ'), (2, 'pass'), (0, 'pass'), (1, '9'), (2, 'pass'), (0, 'K'), (1, 'A'), (2, 'pass'), (0, 'pass'), (1, '4'), (2, 'pass'), (0, '7'), (1, 'T'), (2, 'pass'), (0, 'pass'), (1, '4')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
time 16.641571521759033 89
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['D3 H5 D7 DJ CK DA SA', '', 'H3 H4 D4 S4 C5 H6 

time 17.737720489501953 98
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['H7 C8 S9 DJ', '', 'S3 D3 C3 S4 S7 H8 D8 C9 DT CT CQ SK SA CA DA C2'], 'hand_cards': ['789J', '', '33347889TTQKAAA2'], 'trace': [(0, '3'), (1, 'pass'), (2, 'J'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '2'), (1, 'pass'), (2, 'pass'), (0, '45678'), (1, 'TJQKA'), (2, 'pass'), (0, 'pass'), (1, '66'), (2, 'pass'), (0, 'KK'), (1, 'pass'), (2, 'pass'), (0, 'BR'), (1, 'pass'), (2, 'pass'), (0, '6'), (1, '7'), (2, 'pass'), (0, 'pass'), (1, '99'), (2, 'pass'), (0, 'QQ'), (1, 'pass'), (2, 'pass'), (0, 'J'), (1, '2'), (2, 'pass'), (0, 'pass'), (1, '555'), (2, 'pass'), (0, 'pass'), (1, 'T'), (2, 'pass'), (0, 'pass'), (1, '44')], 'current_player': 2, 'legal_actions': []}
(SelfPlayTest pid=3622) [0 1 1]
time 28.347713470458984 99
(SelfPlayTest pid=3622) {'hand_cards_with_suit': ['DT', '', 'D3 C3 S3 S5 D5 S6 C7 C9 CT HJ SJ DA C2 D2'], 'hand_cards': ['T', '', '33355679TJJA22'], 'trace': [(0, '444QQ'), (1, 'pass'), (2, 'pass')

NameError: name 'reward_history' is not defined

In [ ]:
print(four_res)